In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, Concatenate

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
# Custom function to build hidden combination (i.e. convolution) layers

def create_random_combination_layer(input_layer, combination_size, num_combinations, input_dim):
    outputs = []
    
    for _ in range(num_combinations):
        # First random feature selection
        indices_1 = np.random.choice(input_dim, combination_size, replace=False)
        indices_tensor_1 = tf.constant(indices_1, dtype=tf.int32)
        
        # First feature selection using Lambda layer
        slice_layer_1 = Lambda(
            lambda x: tf.gather(x, indices_tensor_1, axis=1),  # Gather selected features
            output_shape=(combination_size,)
        )(input_layer)
        
        # Second random feature selection (after the first random selection)
        indices_2 = np.random.choice(combination_size, combination_size, replace=False)
        indices_tensor_2 = tf.constant(indices_2, dtype=tf.int32)
        
        # Second feature selection using Lambda layer
        slice_layer_2 = Lambda(
            lambda x: tf.gather(x, indices_tensor_2, axis=1),  # Apply a second feature selection
            output_shape=(combination_size,)
        )(slice_layer_1)

        # Apply Dense layers on the final selected subset
        selected_features = Dense(16, activation='relu')(
            Dense(8, activation='relu')(
                Dense(4, activation='relu')(slice_layer_2)
            )
        )
        outputs.append(selected_features)
    
    # Concatenate the outputs from all the random feature combinations
    return Concatenate()(outputs)

In [5]:
def run_model(train_data_path, test_data_path, is_string_labels=False, label_mapping=None):
    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if is_string_labels:
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(columns=['label']).values
    train_y = train_data['label'].values
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))
    del train_data
    gc.collect()

    # Load and Prepare Test Data
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if is_string_labels:
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))
    del test_data
    gc.collect()

    # Parameters for Random Feature Combination
    num_combinations = 20  # Number of random column combinations
    combination_size = 3   # Number of columns in each combination

    # EarlyStopping Callback (based on training loss)
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 5

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    for run in range(num_runs):
        print(f"Run {run + 1}/{num_runs} started...")

        # Compute class weights for training data
        class_weights = compute_class_weight(
            class_weight='balanced',
            classes=np.unique(train_y.argmax(axis=1)),  # Convert one-hot back to class indices
            y=train_y.argmax(axis=1)
        )

        # Convert class weights to a dictionary
        class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

        # Define the model
        input_layer = Input(shape=(train_X.shape[1],))  # Input shape from the training data
        feature_layer = create_random_combination_layer(input_layer, combination_size, num_combinations, train_X.shape[1])
        hidden_layer = Dense(128, activation='relu')(feature_layer)
        hidden_layer = Dropout(0.5)(hidden_layer)
        output_layer = Dense(test_y.shape[1], activation='softmax')(hidden_layer)
        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Train the model without validation
        history = model.fit(
            train_X, train_y,
            epochs=1000,
            batch_size=int(train_X.shape[0] * 0.01),
            callbacks=[early_stopping],
            class_weight=class_weights_dict,
            verbose=0
        )

        # Evaluate on the test set
        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)

        # Compute metrics
        balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)
        
        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                metrics_storage[label].append(values)

        # Cleanup
        del model
        gc.collect()

    # Average the metrics over all runs
    print("\nAggregated Metrics:")
    for metric, values in metrics_storage.items():
        avg_value = np.mean(values)
        print(f"{metric}: {avg_value:.4f}")

    gc.collect()


In [6]:
run_model("/kaggle/input/ma-datasets/shuttle_train.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

Run 1/5 started...
Run 2/5 started...
Run 3/5 started...
Run 4/5 started...
Run 5/5 started...

Aggregated Metrics:
test_loss: 0.8720
test_accuracy: 0.7597
balanced_accuracy: 0.5565
roc_auc: 0.9117
pr_auc: 0.9062
f2: 0.7661
mcc: 0.4689
0_precision: 0.9435
0_recall: 0.8234
0_f1-score: 0.8765
0_support: 9117.0000
1_precision: 0.3677
1_recall: 0.3800
1_f1-score: 0.2981
1_support: 10.0000
2_precision: 0.3704
2_recall: 0.3118
2_f1-score: 0.1918
2_support: 34.0000
3_precision: 0.4378
3_recall: 0.3626
3_f1-score: 0.3817
3_support: 1781.0000
4_precision: 0.5444
4_recall: 0.9847
4_f1-score: 0.6702
4_support: 653.0000
5_precision: 0.3800
5_recall: 0.5000
5_f1-score: 0.3810
5_support: 2.0000
6_precision: 0.1531
6_recall: 0.5333
6_f1-score: 0.2214
6_support: 3.0000
accuracy: 0.7597
macro avg_precision: 0.4567
macro avg_recall: 0.5565
macro avg_f1-score: 0.4315
macro avg_support: 11600.0000
weighted avg_precision: 0.8410
weighted avg_recall: 0.7597
weighted avg_f1-score: 0.7861
weighted avg_support

In [7]:
run_model("/kaggle/input/ma-datasets/covtype_train.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Run 1/5 started...
Run 2/5 started...
Run 3/5 started...
Run 4/5 started...
Run 5/5 started...

Aggregated Metrics:
test_loss: 1.8141
test_accuracy: 0.2353
balanced_accuracy: 0.2166
roc_auc: 0.5899
pr_auc: 0.4064
f2: 0.2072
mcc: 0.0691
0_precision: 0.1233
0_recall: 0.0333
0_f1-score: 0.0525
0_support: 42368.0000
1_precision: 0.5814
1_recall: 0.3878
1_f1-score: 0.3540
1_support: 56661.0000
2_precision: 0.2861
2_recall: 0.1442
2_f1-score: 0.1355
2_support: 7151.0000
3_precision: 0.0711
3_recall: 0.2379
3_f1-score: 0.0656
3_support: 549.0000
4_precision: 0.0295
4_recall: 0.0476
4_f1-score: 0.0266
4_support: 1899.0000
5_precision: 0.0089
5_recall: 0.0350
5_f1-score: 0.0142
5_support: 3473.0000
6_precision: 0.1372
6_recall: 0.6301
6_f1-score: 0.0829
6_support: 4102.0000
accuracy: 0.2353
macro avg_precision: 0.1768
macro avg_recall: 0.2166
macro avg_f1-score: 0.1045
macro avg_support: 116203.0000
weighted avg_precision: 0.3520
weighted avg_recall: 0.2353
weighted avg_f1-score: 0.2042
weighte

In [8]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("/kaggle/input/ma-datasets/kdd_train.csv", "/kaggle/input/ma-datasets/kdd_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/5 started...
Run 2/5 started...
Run 3/5 started...
Run 4/5 started...
Run 5/5 started...

Aggregated Metrics:
test_loss: 2.8735
test_accuracy: 0.0121
balanced_accuracy: 0.1161
roc_auc: 0.6250
pr_auc: 0.9263
f2: 0.0046
mcc: 0.0574
0_precision: 0.5542
0_recall: 0.0003
0_f1-score: 0.0006
0_support: 194557.0000
1_precision: 0.1094
1_recall: 0.1901
1_f1-score: 0.1389
1_support: 3178.0000
2_precision: 0.0131
2_recall: 0.3848
2_f1-score: 0.0250
2_support: 2496.0000
3_precision: 0.0970
3_recall: 0.2345
3_f1-score: 0.0316
3_support: 2083.0000
4_precision: 0.0086
4_recall: 0.2838
4_f1-score: 0.0158
4_support: 463.0000
5_precision: 0.1851
5_recall: 0.4317
5_f1-score: 0.1933
5_support: 441.0000
6_precision: 0.0037
6_recall: 0.1392
6_f1-score: 0.0071
6_support: 204.0000
7_precision: 0.0141
7_recall: 0.0408
7_f1-score: 0.0210
7_support: 196.0000
8_precision: 0.0000
8_recall: 0.0000
8_f1-score: 0.0000
8_support: 53.0000
9_precision: 0.0001
9_recall: 0.2000
9_f1-score: 0.0002
9_support: 11.0000


In [9]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-datasets/darknet_train.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/5 started...
Run 2/5 started...
Run 3/5 started...
Run 4/5 started...
Run 5/5 started...

Aggregated Metrics:
test_loss: 2.0229
test_accuracy: 0.1707
balanced_accuracy: 0.3161
roc_auc: 0.7242
pr_auc: 0.7833
f2: 0.1696
mcc: 0.0905
0_precision: 0.9757
0_recall: 0.1551
0_f1-score: 0.2504
0_support: 26862.0000
1_precision: 0.0757
1_recall: 0.1883
1_f1-score: 0.0600
1_support: 2657.0000
2_precision: 0.0418
2_recall: 0.5216
2_f1-score: 0.0763
2_support: 908.0000
3_precision: 0.1270
3_recall: 0.1525
3_f1-score: 0.0330
3_support: 522.0000
4_precision: 0.0341
4_recall: 0.3468
4_f1-score: 0.0619
4_support: 293.0000
5_precision: 0.0287
5_recall: 0.0662
5_f1-score: 0.0316
5_support: 269.0000
6_precision: 0.0127
6_recall: 0.1172
6_f1-score: 0.0203
6_support: 116.0000
7_precision: 0.0417
7_recall: 0.5925
7_f1-score: 0.0534
7_support: 53.0000
8_precision: 0.0473
8_recall: 0.7045
8_f1-score: 0.0811
8_support: 44.0000
accuracy: 0.1707
macro avg_precision: 0.1539
macro avg_recall: 0.3161
macro avg